In [ ]:
!pip install biopython gensim umap-learn scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


# Create W2V Model - Rice

4-mers / Vector: 100

In [1]:
kmer_size = 4
vector_size = 100

In [4]:
from Bio import SeqIO
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from joblib import load

def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

fasta_handle = open('osaj43883_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('oryza_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='oryza_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='oryza_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_rice_4mer_100.bin")

# Create W2V Model - Maize

4-mers / Vector: 100

In [3]:
fasta_handle = open('zma10381_genomic_seq.txt', 'r')
fasta_parser = SeqIO.parse(fasta_handle, 'fasta')

with open('maize_corpus.txt', 'w') as corpus_handle:
  for record in fasta_parser:
    record_kmers = circrna_to_kmers(str(record.seq))
    corpus_handle.write(' '.join(record_kmers) + '\n')

w2v_model = Word2Vec(vector_size=vector_size)
w2v_model.build_vocab(corpus_file='maize_corpus.txt')

w2v_model.corpus_count

# Training the w2v model
w2v_model.train(corpus_file='maize_corpus.txt', total_words=w2v_model.corpus_total_words, epochs=1)

# Save the model
w2v_model.save("word2vec_model_maize_4mer_100.bin")

# Create rice dataset W2Vec
4mer / vector: 100

In [5]:
def circrna_to_kmers(circrna_sequence, k=kmer_size):
  kmers = []
  for i in range(0, len(circrna_sequence)-k+1):
    kmer = circrna_sequence[i:i+k]
    kmers.append(kmer)
  return kmers

In [6]:
def circrna_to_vec(circrna_sequence, k=kmer_size):
  vec = np.zeros(vector_size)
  kmers = circrna_to_kmers(circrna_sequence, k=k)
  for kmer in kmers:
    if kmer in w2v_model.wv:
      vec = vec + w2v_model.wv[kmer]
  return vec

In [7]:
w2v_model = Word2Vec.load("word2vec_model_rice_4mer_100.bin")

In [8]:
circ_df = pd.read_excel('rice_db.xlsx')

In [9]:
list = circ_df['seq']

columns = [f'wc_4mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-9-1cc24255fe7b>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_4mer_1,wc_4mer_2,wc_4mer_3,wc_4mer_4,wc_4mer_5,wc_4mer_6,wc_4mer_7,wc_4mer_8,wc_4mer_9,wc_4mer_10,...,wc_4mer_91,wc_4mer_92,wc_4mer_93,wc_4mer_94,wc_4mer_95,wc_4mer_96,wc_4mer_97,wc_4mer_98,wc_4mer_99,wc_4mer_100
0,6.198358,13.380127,-16.500841,8.439034,8.009839,2.741804,-16.308842,9.991552,28.978602,19.888712,...,-6.363861,-36.436143,-8.552242,0.978707,8.992315,1.591115,-15.248491,-30.118378,11.309332,-40.899441
1,141.383675,-4.199321,95.741128,-124.554137,-105.197356,119.876425,172.568755,128.406514,-1.830596,-9.444129,...,-42.182207,-114.801700,-38.580620,20.765011,144.489825,39.080952,7.074321,38.617188,21.643701,-246.901102
2,-14.998783,-33.235679,1.661533,59.858641,58.442054,-61.013836,-40.499265,86.624678,29.806342,40.313869,...,-5.744700,-79.256867,94.005414,104.711629,73.647733,6.227134,66.285729,-58.297799,38.433890,43.095463
3,92.762997,-19.121833,36.625650,-100.414168,-28.890303,-7.369649,134.661739,70.225758,-32.374596,26.881333,...,-44.255107,-50.276061,39.406971,50.541447,79.998565,-36.372162,80.204396,-40.858343,-0.131521,-151.358143
4,-80.943559,7.313482,-13.702572,62.603745,6.005110,-5.844889,-95.822611,34.135532,48.115244,13.693706,...,10.703020,-44.248984,-80.874549,23.750525,-47.672443,51.129628,27.641161,-37.365973,10.743558,42.495435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63043,40.507302,1.399417,17.723197,-52.387915,20.733728,28.557368,46.004313,74.871676,-38.324220,33.736066,...,-58.090967,-29.699576,33.631986,7.004685,9.535375,30.681267,11.525961,-31.919222,-3.529421,-72.659597
63044,-100.745013,-60.068217,-84.938084,-38.465697,-82.872082,-167.124612,-132.321709,76.754234,-36.518780,47.520551,...,46.240108,-54.343659,72.923658,59.667987,-113.834143,134.038868,135.147509,-111.057080,-27.438433,129.410983
63045,-7.923673,-38.948884,-70.902650,-36.467204,-94.696072,-127.169210,-80.456875,50.946420,-17.001513,18.178592,...,87.713503,-86.591337,39.951634,81.590185,-21.856394,80.166703,101.829114,-51.391920,-32.144946,75.667900
63046,49.796811,-45.050070,7.102940,-10.127935,-11.602358,-61.960097,-34.409777,28.674198,-5.602797,-7.890806,...,-27.252398,-14.788173,-25.011356,-4.135251,34.108502,-6.495723,-35.757447,15.082715,-40.087007,-6.458306


In [10]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_4mer_91,wc_4mer_92,wc_4mer_93,wc_4mer_94,wc_4mer_95,wc_4mer_96,wc_4mer_97,wc_4mer_98,wc_4mer_99,wc_4mer_100
0,osa-circ1-OS01T0723400,1:30167620-30167771_+,OS01T0723400,OS01T0723400-01,-,multipleTissue,1,30167620,30167771,+,...,-6.363861,-36.436143,-8.552242,0.978707,8.992315,1.591115,-15.248491,-30.118378,11.309332,-40.899441
1,osa-circ2-OS03T0223400,3:6461672-6462146_-,OS03T0223400,OS03T0223400-01,-,multipleTissue,3,6461672,6462146,-,...,-42.182207,-114.801700,-38.580620,20.765011,144.489825,39.080952,7.074321,38.617188,21.643701,-246.901102
2,osa-circ3-OS11T0210300,11:5715883-5716030_-,OS11T0210300,OS11T0210300-02,-,multipleTissue,11,5715883,5716030,-,...,-5.744700,-79.256867,94.005414,104.711629,73.647733,6.227134,66.285729,-58.297799,38.433890,43.095463
3,osa-circ4-OS02T0200900,2:5631023-5631244_-,OS02T0200900,OS02T0200900-02,-,multipleTissue,2,5631023,5631244,-,...,-44.255107,-50.276061,39.406971,50.541447,79.998565,-36.372162,80.204396,-40.858343,-0.131521,-151.358143
4,osa-circ5-OS05T0494800,5:24302336-24302448_+,OS05T0494800,OS05T0494800-01,-,multipleTissue,5,24302336,24302448,+,...,10.703020,-44.248984,-80.874549,23.750525,-47.672443,51.129628,27.641161,-37.365973,10.743558,42.495435


In [11]:
new_df.to_excel('rice_w2vec_4mer_100_dataset.xlsx', index=False)

# Create maize dataset W2Vec
4mer / vector: 64

In [12]:
w2v_model = Word2Vec.load("word2vec_model_maize_4mer_100.bin")

In [13]:
circ_df = pd.read_excel('maize_db.xlsx')

In [14]:
list = circ_df['seq']

columns = [f'wc_4mer_{v+1}' for v in range(vector_size)]

df_vecs = pd.DataFrame(columns=columns)

for record in list:
  df_vecs = df_vecs._append(
      [
       dict(zip(columns, circrna_to_vec(record)))
       ],
      ignore_index=True)

df_vecs

<ipython-input-14-1cc24255fe7b>:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vecs = df_vecs._append(


,wc_4mer_1,wc_4mer_2,wc_4mer_3,wc_4mer_4,wc_4mer_5,wc_4mer_6,wc_4mer_7,wc_4mer_8,wc_4mer_9,wc_4mer_10,...,wc_4mer_91,wc_4mer_92,wc_4mer_93,wc_4mer_94,wc_4mer_95,wc_4mer_96,wc_4mer_97,wc_4mer_98,wc_4mer_99,wc_4mer_100
0,-2.156234,-48.405569,-21.326676,37.613126,-17.980888,48.563202,34.588956,-52.764769,64.236975,75.861855,...,1.026913,-22.129560,-47.914149,-94.782530,-37.856996,-106.727033,13.994994,-64.121508,51.823108,53.816330
1,37.551796,31.436895,187.444904,78.003859,48.196577,-122.787335,-110.821724,160.123332,-141.102604,-163.083370,...,-160.708138,-97.489841,-327.504184,-41.109195,-245.574397,76.215883,86.785562,281.541121,-70.846933,101.890198
2,-28.773842,-35.159579,-3.119996,-11.927729,18.733864,43.087047,-74.703424,-52.144543,54.483352,-1.697516,...,27.078156,-48.190202,35.510701,-21.535345,-81.037460,-102.812678,-9.679643,-49.920198,-3.947877,11.667154
3,1.440732,-6.820417,-3.777534,26.747614,-20.510598,40.243725,-1.746984,-34.270893,2.905685,52.344864,...,-4.637087,18.606166,-14.330303,-47.478786,7.565250,-38.377078,18.292545,-11.366113,41.998922,29.023675
4,105.825355,-63.646387,-69.079826,43.837618,38.494386,31.793647,-33.195011,-96.378074,81.837274,10.597840,...,-98.346756,-77.201574,4.016222,-61.379828,-111.377859,-121.394497,-112.779163,157.120481,-39.714512,-55.420771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38780,-16.966853,16.666010,-54.337113,-45.849860,33.291685,68.160375,-131.871765,16.698423,51.537891,-0.065865,...,7.776025,-46.170635,71.860866,15.631647,-47.441629,-38.154795,-22.881257,-39.560058,36.872609,-11.785749
38781,33.315083,-48.415534,-49.280748,-18.430471,16.847159,49.250210,-52.037565,-23.413055,21.845662,9.720021,...,-28.553984,-59.877938,3.230594,-31.742325,-60.685089,-59.484940,-13.012177,19.168995,41.204478,-16.061245
38782,-13.814557,31.140061,-161.144531,-63.210241,37.807494,120.497908,-178.310263,16.852059,64.701659,-24.917302,...,-60.392902,75.316563,63.364449,11.578344,5.979118,-58.936093,-158.849585,-83.875667,62.625056,-130.832672
38783,121.526896,-54.526304,-34.732965,-169.268945,159.916457,273.244931,-445.288851,112.603038,46.368602,-382.967092,...,-154.713602,-355.295415,-88.829528,-253.030870,-296.983902,-93.084420,156.775177,29.937569,-50.366545,0.001963


In [15]:
new_df = pd.concat([circ_df, df_vecs], axis=1)

new_df.head()

,circName,circID,gene,isoform,stress,tissue,chr,start,end,strand,...,wc_4mer_91,wc_4mer_92,wc_4mer_93,wc_4mer_94,wc_4mer_95,wc_4mer_96,wc_4mer_97,wc_4mer_98,wc_4mer_99,wc_4mer_100
0,zma-circ1-Zm00001d002325,2:10317309-10317467_-,Zm00001d002325,Zm00001d002325_T001,-,multipleTissue,2,10317309,10317467,-,...,1.026913,-22.129560,-47.914149,-94.782530,-37.856996,-106.727033,13.994994,-64.121508,51.823108,53.816330
1,zma-circ2-Zm00001d038675,6:162376852-162378246_+,Zm00001d038675,Zm00001d038675_T004,-,multipleTissue,6,162376852,162378246,+,...,-160.708138,-97.489841,-327.504184,-41.109195,-245.574397,76.215883,86.785562,281.541121,-70.846933,101.890198
2,zma-circ3-Zm00001d038163,6:150032431-150032595_+,Zm00001d038163,Zm00001d038163_T001,-,multipleTissue,6,150032431,150032595,+,...,27.078156,-48.190202,35.510701,-21.535345,-81.037460,-102.812678,-9.679643,-49.920198,-3.947877,11.667154
3,zma-circ4-Zm00001d049552,4:34381638-34381747_-,Zm00001d049552,Zm00001d049552_T004,-,multipleTissue,4,34381638,34381747,-,...,-4.637087,18.606166,-14.330303,-47.478786,7.565250,-38.377078,18.292545,-11.366113,41.998922,29.023675
4,zma-circ5-Zm00001d032567,1:230724608-230725226_-,Zm00001d032567,Zm00001d032567_T001,-,multipleTissue,1,230724608,230725226,-,...,-98.346756,-77.201574,4.016222,-61.379828,-111.377859,-121.394497,-112.779163,157.120481,-39.714512,-55.420771


In [16]:
new_df.to_excel('maize_w2vec_4mer_100_dataset.xlsx', index=False)